In [1]:
pip install PyWavelets pytorch_wavelets

Note: you may need to restart the kernel to use updated packages.


In [2]:
import math
import torch
from pytorch_wavelets import DWTForward, DWTInverse


class Basicconv(torch.nn.Module):

    def __init__(self, inchannel, outchannel, groups=1):
        super().__init__()
        self.conv = torch.nn.Conv2d(
            inchannel, outchannel, kernel_size=3, padding=1, groups=groups)
        self.act = torch.nn.PReLU()
        self.nom = torch.nn.GroupNorm(groups, outchannel)

    def forward(self, x):
        x = self.conv(x)
        x = self.nom(x)
        x = self.act(x)
        return x


class Bottleneck(torch.nn.Module):
    def __init__(self, inchannel, outchannel, groups=1):
        super().__init__()
        self.conv = torch.nn.Conv2d(inchannel, outchannel, kernel_size=1, groups=groups)

    def forward(self, x):
        x = self.conv(x)
        return x


class DCRblock(torch.nn.Module):
    def __init__(self, inchannel, groups=1):
        super().__init__()
        self.conv1 = Basicconv(inchannel, inchannel//2, groups=groups)
        self.conv2 = Basicconv(inchannel//2 + inchannel, inchannel//2)
        self.conv3 = Basicconv(2 * inchannel, inchannel)

    def forward(self, x):
        x1 = self.conv1(x)
        x2 = torch.cat([x, x1], dim=1)
        x2 = self.conv2(x2)
        x2 = torch.cat([x1, x2, x], dim=1)
        x3 = self.conv3(x2)
        x = x3 + x
        return x


class finalneck(torch.nn.Module):
    def __init__(self, inchannel, outchannel):
        super().__init__()
        self.conv = torch.nn.Conv2d(
            inchannel, outchannel, kernel_size=3, padding=1)

    def forward(self, x):
        x = self.conv(x)
        return x


class finalcombine(torch.nn.Module):
    def __init__(self, inchannel, wave="Haar", groups=False):
        super().__init__()
        self.groups = groups
        self.c00 = Basicconv(inchannel=inchannel,  outchannel=160)
        self.DWT = DWTForward(1, wave, "symmetric")
        self.IDWT = DWTInverse(wave, "symmetric")
        self.c1 = Bottleneck(inchannel=320, outchannel=320)
        self.d1 = Bottleneck(inchannel=320, outchannel=320)
        self.c2 = DCRblock(inchannel=320)
        self.d2 = DCRblock(inchannel=320)
        self.c3 = DCRblock(inchannel=320)
        self.d3 = DCRblock(inchannel=320)
        self.c4 = finalneck(inchannel=320, outchannel=3)
        self.d4 = finalneck(inchannel=320, outchannel=3)
        self.c11 = Basicconv(
            inchannel=inchannel * 4,  outchannel=256, groups=4 if groups else 1)
        self.c12 = Bottleneck(inchannel=512, outchannel=512)
        self.c13 = DCRblock(inchannel=512)
        self.c14 = finalneck(inchannel=512, outchannel=640)
        self.c21 = Basicconv(inchannel=inchannel * 4 * 4,
                             outchannel=256, groups=4 if groups else 1)
        self.c22 = Bottleneck(inchannel=512, outchannel=512)
        self.c23 = DCRblock(inchannel=512)
        self.c24 = finalneck(inchannel=512, outchannel=1024)
        self.c31 = Basicconv(inchannel=inchannel * 4 * 4 * 4,
                             outchannel=256, groups=4 if groups else 1)
        self.c32 = DCRblock(inchannel=256, groups=4 if groups else 1)
        self.c33 = finalneck(inchannel=256, outchannel=1024)

    def forward(self, x):
        x00 = self.c00(x)
        b, c, w, h = x.shape
        w = w // 2
        h = h // 2
        xl, xh = self.DWT(x)
        xh = xh[0][:, :, :, :w, :h]
        xl = xl[:, :, :w, :h]
        b, c, _, w, h = xh.shape
        xh = xh.reshape(b, 3*c, w, h)
        x1 = torch.cat([xl, xh], dim=1)

        x11 = self.c11(x1)
        b, c, w, h = x1.shape
        w = w // 2
        h = h // 2
        xl, xh = self.DWT(x1)
        xh = xh[0][:, :, :, :w, :h]
        xl = xl[:, :, :w, :h]
        b, c, _, w, h = xh.shape
        xh = xh.reshape(b, 3*c, w, h)
        x2 = torch.cat([xl, xh], dim=1)

        x21 = self.c21(x2)
        b, c, w, h = x2.shape
        w = w // 2
        h = h // 2
        xl, xh = self.DWT(x2)
        xh = xh[0][:, :, :, :w, :h]
        xl = xl[:, :, :w, :h]
        b, c, _, w, h = xh.shape
        xh = xh.reshape(b, 3*c, w, h)
        x3 = torch.cat([xl, xh], dim=1)

        x31 = self.c31(x3)
        x32 = self.c32(x31)
        x33 = self.c33(x32)
        c = x33.shape[1] // 4
        xl = x33[:, :c]
        xh = x33[:, c:]
        b, c, w, h = xl.shape
        xh = [xh.reshape(b, c, 3, w, h)]
        w = w * 2
        h = h * 2
        xi = self.IDWT((xl, xh))
        xi = torch.nn.functional.pad(
            xi, (0, h-xi.shape[2], 0, w-xi.shape[3]), mode="reflect")
        x21 = torch.cat([x21, xi], dim=1)
        x22 = self.c22(x21)
        x23 = self.c23(x22)
        x24 = self.c24(x23)
        c = x24.shape[1] // 4
        xl = x24[:, :c]
        xh = x24[:, c:]
        b, c, w, h = xl.shape
        xh = [xh.reshape(b, c, 3, w, h)]
        w *= 2
        h *= 2
        xi = self.IDWT((xl, xh))
        xi = torch.nn.functional.pad(
            xi, (0, h-xi.shape[2], 0, w-xi.shape[3]), mode="reflect")
        x11 = torch.cat([x11, xi], dim=1)
        x12 = self.c12(x11)
        x13 = self.c13(x12)
        x14 = self.c14(x13)
        c = x14.shape[1] // 4
        xl = x14[:, :c]
        xh = x14[:, c:]
        b, c, w, h = xl.shape
        xh = [xh.reshape(b, c, 3, w, h)]
        w *= 2
        h *= 2
        xi = self.IDWT((xl, xh))
        xi = torch.nn.functional.pad(
            xi, (0, w - xi.shape[2], 0, h - xi.shape[3]), mode="reflect")
        x01 = torch.cat([x00, xi], dim=1)
        xc1 = self.c1(x01)
        xc2 = self.c2(xc1) + xc1
        xc3 = self.c3(xc2) + xc2
        xc4 = self.c4(xc3)
        xc4 = torch.nn.functional.tanh(xc4)
        xd1 = self.d1(x01)
        xd2 = self.d2(xd1) + xd1
        xd3 = self.d3(xd2) + xd2
        xd4 = self.d4(xd3)
        xd4 = x - xd4
        output = (xd4 + xc4)/2
        return output

In [3]:
from utils import train, test
from BSD import BSDDataset
import numpy as np

torch.manual_seed(4623)
torch.cuda.manual_seed(4623)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

image_size = 256
epochs = 100
batch_size = 4
lr = 1e-4
criterion = torch.nn.MSELoss()

base_dir=""

train_set = BSDDataset(base_dir=base_dir, split="train")
test_set = BSDDataset(base_dir=base_dir, split="test")


def rgb_to_ycbcr(img):
    M = torch.tensor([[ 0.2990,     0.5870,     0.1140    ],
                      [-0.168736, -0.331264,    0.5       ],
                      [ 0.5,      -0.418688,   -0.081312  ]], dtype=img.dtype, device=img.device)
    bias = torch.tensor([0.0, 0.5, 0.5], dtype=img.dtype, device=img.device).view(3, 1, 1)
    ycbcr = torch.einsum('bchw,mc->bmhw', img, M) + bias
    return ycbcr

def ycbcr_to_rgb(ycbcr):
    M_inv = torch.tensor([[1.0,  0.0,       1.402],
                          [1.0, -0.344136, -0.714136],
                          [1.0,  1.772,    0.0     ]], dtype=ycbcr.dtype, device=ycbcr.device)
    bias = torch.tensor([0.0, -0.5, -0.5], dtype=ycbcr.dtype, device=ycbcr.device).view(3, 1, 1)
    rgb = torch.einsum('bchw,mc->bmhw', ycbcr + bias, M_inv)
    return rgb

def compute_loss(model, images, noise_range=(1, 60), residual=False, luminance=False):
    noise_level = torch.randint(noise_range[0], noise_range[1], (1,)).item()
    noisy_images = images + (noise_level / 255.0) * torch.randn_like(images)
    images = images.to(device)  # move to GPU
    noisy_images = torch.clamp(noisy_images, 0, 1)
    noisy_images = noisy_images.to(device)
    if luminance:
        images = rgb_to_ycbcr(images)[:, :1]
        noisy_images = rgb_to_ycbcr(noisy_images)[:, :1]
    outputs = model(noisy_images)  # forward
    outputs = outputs
    if residual:
        loss = criterion(outputs, noisy_images - images)
        return loss
    loss = criterion(outputs, images)
    return loss


def denoise(model, noisy_img, residual=False, luminance=False):
    if luminance:
        target_img = rgb_to_ycbcr(noisy_img)
        noisy_img = target_img[:, :1]
    if residual:
        outputs = noisy_img - model(noisy_img)
    else:
        outputs = model(noisy_img)  # forward
    if luminance:
        outputs = torch.cat([outputs, target_img[:, 1:]], dim=1)
        outputs = ycbcr_to_rgb(outputs)
    outputs = torch.clamp(outputs, 0, 1)
    return outputs

In [4]:
def experiment(model_name, residual=False, luminance=False, groups=False):
    if luminance:
        model = finalcombine(inchannel=1, groups=groups).to(device)
    else:
        model = finalcombine(inchannel=3, groups=groups).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    train(model, optimizer, epochs, train_set, test_set, batch_size, model_name, compute_loss=compute_loss, residual=residual, luminance=luminance)
    for noise_level in [10, 25, 50]:
        test(model, test_set, batch_size, model_name, noise_level, denoise=denoise, residual=residual, luminance=luminance)

All Combined

In [ ]:
model_name = "DSWN"
residual = True
luminance = True
groups=True

experiment(model_name, residual=residual, luminance=luminance, groups=groups)

DSWN


  0%|          | 0/100 [00:00<?, ?it/s]

Ablation

In [ ]:
model_name = "DSWN-no-residual"
residual = False
luminance = True
groups=True

experiment(model_name, residual=residual, luminance=luminance, groups=groups)

In [ ]:
model_name = "DSWN-no-luminance"
residual = True
luminance = False
groups=True

experiment(model_name, residual=residual, luminance=luminance, groups=groups)

In [ ]:
model_name = "DSWN-no-groups"
residual = True
luminance = True
groups=False

experiment(model_name, residual=residual, luminance=luminance, groups=groups)